<a href="https://colab.research.google.com/github/dylanwalker/BA865/blob/master/BA865_Lecture_07_Exercise_Solutions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
import pandas as pd
import numpy as np

# Define a simple NN and train it to predict music genre on spotify data

For this exercise, we'll use a dataset from Spotify containing music attributes such as bpm, energy, and danceability. It has 45 songs that belong to either the 'rock' genre or the 'dance pop' genre. For this example, as above, we won't bother to split the data into training and test sets (again, the data here is pretty small).

In [0]:
# Read the data
musicFile = 'https://raw.githubusercontent.com/dylanwalker/BA865/master/datasets/spotify.csv'
musicData = pd.read_csv(musicFile, encoding='latin-1')
musicData.head()

The first step is to define the dataset. 

Make a new column in the dataframe called 'genre_class'. Set it to be 0 for 'rock' and 1 for 'dance pop' (these are the only genres in the dataset).

We need to get our pandas data into torch tensors. One way to do this is to first convert them to numpy arrays and then call `torch.from_numpy()` on the numpy arrays. We'll want to set the `dtype=float32` when we call `np.array()` on the pandas columns.

Define the musical attributes ('bpm' through 'speechiness') as a pytorch tensor called `features`.

Define a pytorch tensor called `targets`  based on the column `genre_class`.



In [0]:
# INSERT YOUR CODE HERE
musicData['genre_class'] = 0
musicData.loc[musicData['top genre']=='dance pop','genre_class'] = 1
features = torch.from_numpy(np.array(musicData.iloc[:,3:10], dtype='float32'))
targets = torch.from_numpy(np.array(musicData['genre_class'], dtype='float32'))

In [0]:
targets
features

Now create a TensorDataSet from `features` and `targets` called `train_ds`.

Also create a DataLoader called `train_dl` from `train_ds` using `batch_size=10` and `shuffle=True`. 

In [0]:
from torch.utils.data import TensorDataset, DataLoader
# INSERT YOUR CODE HERE
train_ds = TensorDataset(features,targets)
batch_size = 10
train_dl = DataLoader(train_ds, batch_size, shuffle=True)

Note that we can see the shapes of tensors drawn from the dataloader with our given batch size:

In [0]:
x1,y1=next(iter(train_dl)) # pull a single x,y from the data loader
print(x1.shape)
print(y1.shape)
print(x1)
print(y1)

In [0]:
y1.view(-1,1)

If we wanted to pass these through a loss function, we would likely have to alter the shape of the `y`, to something that our loss function can understand. We can do this with a `.view(-1,1)` operation, which will make it a tensor with 90 rows and 1 column (instead of a 1d tensor), like so:

In [0]:
print(x1,y1.view(-1,1))
#y1
#y1.view(-1,1)
#print(y1.view(-1,1).shape) # this will output a shape of [batch_size,1]

For our neural network model we'll use a single fully connected layer (`torch.nn.Linear()`), followed by a sigmoid activation function (`torch.nn.Sigmoid()`). 

Note that `torch.nn.Linear()` takes two arguments that specify the size of the input and output. You'll have to figure out what these should be based on the data. Remember that we want to connect all the features to a single output thta is then passed through the activation function. `toch.nn.Sigmoid()` does not need any arguments.

We can create a model object from these two layers by passing them as arguments to `torch.nn.Sequential()`. This will make a neural network that connects the output of each layer to the input of the next.

By using the sigmoid function, the output of the fully connected layers is converted into a value between 0 and 1. If the converted value is larger than 0.5, we will interpret this as the model predicting the genre of a music to be 'dance pop' (since we set `genre_class=1` for 'dance pop'. Otherwise, the model predicts the genre to be 'rock' music.

We'll set the random seed of pytorch manually, using `torch.manual_seed(0)` just to ensure that we are all working on the same shuffled sequence. This step is not needed in general when you train a NN with shuffling, but just included to make sure we're all on the same page.

In [0]:
torch.manual_seed(0)
# model = FILL IN YOUR CODE HERE
model = torch.nn.Sequential(torch.nn.Linear(7, 1), torch.nn.Sigmoid())

Now instantiate an SGD optimizer using `torch.optim.SGD()`. We can use a learning rate of `lr=1e-4`.

For the loss function, we'll use the binary cross entropy loss, `torch.nn.BCELoss()` with the argument `reduction='mean'`. BCELoss is often used in logistic regression.

In [0]:
# optimizer = FILL IN YOUR CODE HERE
# loss_function = FILL IN YOUR CODE HERE
lr_rate = 1e-4
optimizer = torch.optim.SGD(model.parameters(), lr=lr_rate)
loss_function = torch.nn.BCELoss(reduction='mean') # binary cross entropy

Now write the code for training the model over 1000 epochs. You can look back at my `fit()` function defined above. You don't need to write this as a function, but can instead just write the code directly.

The model will be trained for 1000 epochs (i.e., you need to loop over epochs).

When you call `loss_function`, on the predictions (`pred`) and the targets (`yb`), you will need to adjust the shape of the targets, as explained above.

We can calculate a running_loss for each epoch by setting `running_loss=0` before looping over the items in `train_dl`.  We can add up the contribution from each batch to the running loss by adding a line `running_loss+=loss.item()` after calling `loss.backward()`.

Finally, we may like to print out the loss for some epochs (there are 1000 epochs, so its overkill to print out the loss for each one). Instead we can print out the loss for only certain values of the epoch -- for example when the epoch divided by 100 has a remainder of 99, or `epoch%100==99`. When this occurs, print out the average loss for that epoch, `running_loss/len(train_dl)`. 

You can also try your hand at calculating either the average training accuracy for an epcoh or the "last batch training accuracy" of the model at these particular epochs.

In [0]:
# FILL IN YOUR CODE HERE
n_epochs = 500
model.train()
for epoch in range(n_epochs):
  pass # replace this with your code 

n_epochs = 300
model.train() 
for epoch in range(n_epochs):
    running_loss = 0.0
    for X, y in train_dl:
        optimizer.zero_grad()
        pred = model(X)
        loss = loss_function(pred, y.view(-1,1))
        loss.backward()
        running_loss += loss.item()
        optimizer.step()
    if epoch%10==9:
        pred = np.array([0 if x<=0.5 else 1 for x in pred])
        loss_value = round(running_loss/len(train_dl), 2)
        print(f'Epoch: {epoch+1: <{3}}, Loss: {loss_value: <{3}}')

Finally, when the model has been fully trained, let's look at a prediction for a particular song. We'll predict the genre of Ariana Grande's "no tears left to cry." This song is classified as 'dance pop' in the Spotify. Here's the music video:

In [0]:
from IPython.display import YouTubeVideo
YouTubeVideo('ffxKSjUwKdU',width=400,height=300)

This particular song has the features (bpm, energy, danceability, liveness, valence, acousticness, speechiness): [122, 71, 70, 29, 35, 4, 6]

Get the prediction for this particular song by passing the appropriate input into the model to get the output. How do you interpret this output?

In [0]:
# INSERT YOUR CODE HERE
no_tears_left_to_cry = torch.tensor([122, 71, 70, 29, 35, 4, 6]).float()
print(model(no_tears_left_to_cry))

The model returns 0.56 for "no tears left to cry," so it classifies the song as "dance pop."